In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
class Head(nn.Module):
  """one head of self-attention"""

  def __init__(self, head_size):
      super().__init__()
      self.key = nn.Linear(n_embd, head_size, bias=False)
      self.query = nn.Linear(n_embd, head_size, bias=False)
      self.value = nn.Linear(n_embd, head_size, bias=False)
      self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

      self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)
    # compute attention score ("affinities")
    wei = q @ k.transpose(-2, -1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
    wei = F.softmax(wei, dim=-1)# (B, T, T)
    wei = self.dropout(wei)
    # perform the weighted aggregation of the values
    v = self.value(x)
    out = wei @ v # (B, T, C)
    return out

In [3]:
n_embd = 5
mem_size = 7

token = torch.randn(1, n_embd)
m = torch.randn(1, mem_size)

In [28]:
key = nn.Linear(n_embd, n_embd, bias=False)
query = nn.Linear(mem_size, n_embd, bias=False)
value = nn.Linear(mem_size, n_embd, bias=False)
tril = torch.tril(torch.ones(n_embd, n_embd))

In [41]:
k = key(token) # (1, n_embd)
q = query(m) # (1, n_embd)

wei = q.T @ k * n_embd**-0.5 # (n_embd, n_embd)
wei = wei.masked_fill(tril[:n_embd, :n_embd] == 0, float('-inf')) # (n_embd, n_embd)
wei = F.softmax(wei, dim=-1)# (n_embd, n_embd)

v = value(m) # (1, n_embd)
out = v @ wei # (1, n_embd)
out

tensor([[ 0.0211, -0.3923, -0.0543, -0.1658, -0.0071]], grad_fn=<MmBackward0>)

In [42]:
class Head(nn.Module):
    def __init__(self, head_size, m_size):
        super().__init__()
        self.head_size = head_size
        
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(m_size, head_size, bias=False)
        self.value = nn.Linear(m_size, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(head_size, head_size)))
    
    def forward(self, token, memory):
        k = key(token) # (1, head_size)
        q = query(memory) # (1, head_size)

        wei = q.T @ k * self.head_size**-0.5 # (head_size, head_size)
        wei = wei.masked_fill(tril[:self.head_size, :self.head_size] == 0, float('-inf')) # (head_size, head_size)
        wei = F.softmax(wei, dim=-1)# (head_size, head_size)

        v = value(memory) # (1, head_size)
        out = v @ wei # (1, head_size)
        return out

In [43]:
class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size, m_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, m_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out